### Loading the model

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("s-nlp/russian_toxicity_classifier")
model = AutoModelForSequenceClassification.from_pretrained("s-nlp/russian_toxicity_classifier")

### I did the installing part partly here, partly with conda. Holy shit it's so slow

In [9]:
import sys
!{sys.executable} -m pip install onnx onnxconverter-common

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for onnxconverter-common from https://files.pythonhosted.org/packages/6d/6a/9ed9fd4da34cb41fda35bc5cc9e990c605689db7de63ed84fedbca5a77f6/onnxconverter_common-1.14.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for protobuf>=3.20.2 from https://files.pythonhosted.org/packages/8b/e6/2a47ce2eba1aaf287380a44270da897ada03d118a55c19595ec7b4f0831f/protobuf-3.20.2-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/84.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/84.5 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/84.5 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/84.5 kB ? eta -:--:--
   ------------------- -------------------- 41.0/84.5 kB 245.8 kB/s eta 0:00:01
   ---------------------------------------- 84.5/84.5 kB 431.6 kB/s eta 0:00:00
   ---------------------------------

ERROR: Could not install packages due to an OSError: [WinError 5] Отказано в доступе: 'C:\\Users\\ulyan\\AppData\\Roaming\\Python\\Python311\\site-packages\\google\\~upb\\_message.pyd'
Check the permissions.



In [4]:
import torch
import onnx
import os

PREFIX_MODELS = "triton_inference//model_repository"

### Saving models 
- pth (original)
- onnx_fp32
- onnx_fp16
- trt_fp16

Actually you might notice that I'm saving in simply folders, and I had to manually move all thiose files to 1/ folder)

In [6]:
# orig
model.save_pretrained(f"{PREFIX_MODELS}//pth")
tokenizer.save_pretrained(f"{PREFIX_MODELS}//pth")

('triton_inference//model_repository//pth\\tokenizer_config.json',
 'triton_inference//model_repository//pth\\special_tokens_map.json',
 'triton_inference//model_repository//pth\\vocab.txt',
 'triton_inference//model_repository//pth\\added_tokens.json',
 'triton_inference//model_repository//pth\\tokenizer.json')

In [7]:
# onnx_fp32
onnx_path = f"{PREFIX_MODELS}/onnx_fp32/model.onnx"

sequence_length = 128
dummy_input = tokenizer("Это пример текста.", padding='max_length', truncation=True, max_length=sequence_length, return_tensors="pt")
input_names = ["input_ids", "attention_mask"]
output_names = ["output"]
dynamic_axes = {
    "input_ids": {0: "batch_size", 1: "sequence_length"},
    "attention_mask": {0: "batch_size", 1: "sequence_length"},
    "output": {0: "batch_size"}
}

model.eval()
torch.onnx.export(
    model,
    (dummy_input["input_ids"], dummy_input["attention_mask"]),
    onnx_path,
    opset_version=13,
    input_names=input_names,
    output_names=output_names,
    dynamic_axes=dynamic_axes,
    do_constant_folding=True,
    verbose=False
)

onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)

In [12]:
from onnxconverter_common import float16

onnx_fp32_path = f"{PREFIX_MODELS}/onnx_fp32/model.onnx"
onnx_fp16_path = f"{PREFIX_MODELS}/onnx_fp16/model.onnx"

onnx_model_fp16 = float16.convert_float_to_float16(onnx_model)
onnx.save(onnx_model_fp16, onnx_fp16_path)

D:\E\Downloads\Anaconda\Lib\site-packages\onnxconverter_common\float16.py:43: UserWarning: the float32 number 3.5592728764299864e-09 will be truncated to 1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(pos_min, min_positive_val))
D:\E\Downloads\Anaconda\Lib\site-packages\onnxconverter_common\float16.py:53: UserWarning: the float32 number -8.079577284991757e-11 will be truncated to -1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(neg_max, -min_positive_val))
D:\E\Downloads\Anaconda\Lib\site-packages\onnxconverter_common\float16.py:43: UserWarning: the float32 number 4.6042714529903606e-08 will be truncated to 1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(pos_min, min_positive_val))
D:\E\Downloads\Anaconda\Lib\site-packages\onnxconverter_common\float16.py:43: UserWarning: the float32 number 6.83335201756563e-08 will be truncated to 1e-07
  warnings.warn("the float32 number {} will be truncated to {}"

D:\E\Downloads\Anaconda\Lib\site-packages\onnxconverter_common\float16.py:43: UserWarning: the float32 number 9.080340390710262e-08 will be truncated to 1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(pos_min, min_positive_val))
D:\E\Downloads\Anaconda\Lib\site-packages\onnxconverter_common\float16.py:43: UserWarning: the float32 number 5.124388025024018e-08 will be truncated to 1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(pos_min, min_positive_val))
D:\E\Downloads\Anaconda\Lib\site-packages\onnxconverter_common\float16.py:53: UserWarning: the float32 number -1.296058727007221e-08 will be truncated to -1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(neg_max, -min_positive_val))
D:\E\Downloads\Anaconda\Lib\site-packages\onnxconverter_common\float16.py:43: UserWarning: the float32 number 1.1694718971000384e-08 will be truncated to 1e-07
  warnings.warn("the float32 number {} will be truncated to {}"

D:\E\Downloads\Anaconda\Lib\site-packages\onnxconverter_common\float16.py:43: UserWarning: the float32 number 6.032205401851343e-09 will be truncated to 1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(pos_min, min_positive_val))
D:\E\Downloads\Anaconda\Lib\site-packages\onnxconverter_common\float16.py:53: UserWarning: the float32 number -1.2621200085050077e-08 will be truncated to -1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(neg_max, -min_positive_val))
D:\E\Downloads\Anaconda\Lib\site-packages\onnxconverter_common\float16.py:43: UserWarning: the float32 number 3.7797423857455215e-08 will be truncated to 1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(pos_min, min_positive_val))
D:\E\Downloads\Anaconda\Lib\site-packages\onnxconverter_common\float16.py:53: UserWarning: the float32 number -2.1835933949887476e-08 will be truncated to -1e-07
  warnings.warn("the float32 number {} will be truncated to

D:\E\Downloads\Anaconda\Lib\site-packages\onnxconverter_common\float16.py:43: UserWarning: the float32 number 7.47634398834407e-09 will be truncated to 1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(pos_min, min_positive_val))
D:\E\Downloads\Anaconda\Lib\site-packages\onnxconverter_common\float16.py:53: UserWarning: the float32 number -8.269574891528464e-08 will be truncated to -1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(neg_max, -min_positive_val))
D:\E\Downloads\Anaconda\Lib\site-packages\onnxconverter_common\float16.py:50: UserWarning: the float32 number -3.4028234663852886e+38 will be truncated to -10000.0
  warnings.warn("the float32 number {} will be truncated to {}".format(neg_min, -max_finite_val))
D:\E\Downloads\Anaconda\Lib\site-packages\onnxconverter_common\float16.py:43: UserWarning: the float32 number 9.999999960041972e-13 will be truncated to 1e-07
  warnings.warn("the float32 number {} will be truncated to {

In [ ]:
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import os

TRT_LOGGER = trt.Logger()

def build_engine(engine_file_path, fp16_mode=True):
    with (trt.Builder(TRT_LOGGER) as builder, 
          builder.create_builder_config() as config, 
          builder.create_network(1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)) as network,
          trt.OnnxParser(network, TRT_LOGGER) as parser):
        config.max_workspace_size = 1 << 30  # 1G
        if fp16_model:
            config.flags |= 1 << int(trt.BuilderFlag.FP16)

        engine = builder.build_serialized_network(network, config)
        if engine is None:
            print("ERROR: Could not build engine.")
            return None

        with open(engine_file_path, "wb") as f:
            f.write(engine)
        return engine

trt_engine_path = f"{PREFIX_MODELS}/trt_fp16/model.plan"

engine = build_engine(onnx_fp16_path, trt_engine_path)
if engine:
    print(f"TensorRT engine saved to {trt_engine_path}")
else:
    print("TensorRT engine build failed.")

Then I manually made configs and so on...

## Training

I am working on windows so no gpu in triton sorry
Also as one of the comments claimed - I used a [different dataset](https://huggingface.co/datasets/AlexSham/Toxic_Russian_Comments)

In [14]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, auc, precision_score
import matplotlib.pyplot as plt
from tqdm import tqdm

splits = {'train': 'train.jsonl', 'test': 'test.jsonl'}
df_train = pd.read_json("hf://datasets/AlexSham/Toxic_Russian_Comments/" + splits["train"], lines=True)
df_test = pd.read_json("hf://datasets/AlexSham/Toxic_Russian_Comments/" + splits["test"], lines=True)

D:\E\Downloads\Anaconda\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [15]:
df_train.head()

,text,label
0,"видимо в разных регионах называют по разному ,...",0
1,"понятно что это нарушение правил, писать капсл...",1
2,"какие классные, жизненные стихи....",0
3,а и правда-когда его запретили?...,0
4,в соленой воде вирусы живут .ученые изучали со...,0


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
device

device(type='cpu')

In [44]:
model.eval()
all_logits_list = []
batch_size = 32

for i in tqdm(range(0, len(df_train), batch_size), desc="Processing train..."):
    batch_texts = df_train.loc[i:min(i + batch_size, len(df_train)), "text"].to_list()
    inputs = tokenizer(batch_texts, padding='max_length', truncation=True, max_length=128, return_tensors="pt")
    inputs = inputs.to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits.cpu().numpy()
    all_logits_list.append(logits)

all_logits = np.concatenate(all_logits_list, axis=0)

Processing train...:   0%|                                                         | 8/6984 [00:45<10:56:17,  5.64s/it]


KeyboardInterrupt: 

In [ ]:
probabilities = np.exp(all_logits[:, 1]) / np.sum(np.exp(all_logits), axis=1)

def find_optimal_threshold(probabilities, labels, target_precision=0.9):
    thresholds = np.sort(probabilities)
    for threshold in thresholds:
        predictions = (probabilities >= threshold).astype(int)
        print(len(labels), len(predictions))
        precision = precision_score(labels, predictions)
        if precision >= target_precision:
            return threshold
    return 0.5

optimal_threshold = find_optimal_threshold(probabilities, df_train["label"], target_precision=0.9)
print(f"Optimal threshold with 0.9 precision: {optimal_threshold}")

In [ ]:
roc_auc = roc_auc_score(df_train["label"], probabilities)
print(f"ROC AUC: {roc_auc}")

fpr, tpr, thresholds = roc_curve(test_labels, probabilities)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.show()

In [ ]:
closest_idx = np.argmin(np.abs(thresholds - optimal_threshold))
plt.scatter(fpr[closest_idx], tpr[closest_idx], color='red', label=f'Threshold = {optimal_threshold:.2f}')
plt.legend(loc="lower right")
plt.show()

precision, recall, thresholds = precision_recall_curve(test_labels, probabilities)
pr_auc = auc(recall, precision)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f'PR curve (area = {pr_auc:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])

recall_threshold_index = np.argmin(np.abs(thresholds - optimal_threshold))

plt.scatter(recall[recall_threshold_index], precision[recall_threshold_index], color='red', label=f'Threshold = {optimal_threshold:.2f}')
plt.legend()
plt.show()